In [2]:
!pip install datasets

You are using pip version 10.0.1, however version 23.1.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [1]:
# 下載數據集
from datasets import load_dataset

# 下載IMDb情感分類數據集
dataset = load_dataset('imdb')

# 下載AG新聞分類數據集
dataset = load_dataset('ag_news')

# 下載SQuAD問答數據集
dataset = load_dataset('squad')

# 下載預訓練模型
from transformers import AutoModel, AutoTokenizer

# 下載BERT模型和tokenizer
model = AutoModel.from_pretrained('bert-base-uncased')
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# 下載GPT-2模型和tokenizer
model = AutoModel.from_pretrained('gpt2')
tokenizer = AutoTokenizer.from_pretrained('gpt2')

# 下載RoBERTa模型和tokenizer
model = AutoModel.from_pretrained('roberta-base')
tokenizer = AutoTokenizer.from_pretrained('roberta-base')

Found cached dataset imdb (C:/Users/user/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)


  0%|          | 0/3 [00:00<?, ?it/s]

Found cached dataset ag_news (C:/Users/user/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)


  0%|          | 0/2 [00:00<?, ?it/s]

Found cached dataset squad (C:/Users/user/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


  0%|          | 0/2 [00:00<?, ?it/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
from transformers import BertForSequenceClassification, BertTokenizer, AdamW

# 加載BERT模型和tokenizer
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# 下載IMDb數據集
dataset = load_dataset('imdb')

# 加載預訓練模型和tokenizer
model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 準備訓練數據集
train_texts = dataset['train']['text']
train_labels = dataset['train']['label']

# 使用tokenizer將文本轉換為input_ids和attention_mask
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
train_encodings['labels'] = train_labels

# 轉換為PyTorch數據集格式
import torch
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings['input_ids'])

train_dataset = MyDataset(train_encodings)
test_dataset = MyDataset(train_encodings)
# 加載數據集並轉換為BERT格式
#dataset = load_dataset('imdb')
#train_dataset = dataset['train']
#test_dataset = dataset['test']

# 定義訓練流程
batch_size = 32
learning_rate = 2e-5
optimizer = AdamW(model.parameters(), lr=learning_rate)

# 訓練模型
model.train()
num_epochs = 10  # 設置總共的訓練迭代次數
for epoch in range(num_epochs):
    for batch in train_dataset:
        print(batch)
        #input_ids = batch['input_ids']
        #attention_mask = batch['attention_mask']
        #labels = batch['labels']
        #outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        inputs = batch
        outputs = model(inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

# 評估模型
model.eval()
for batch in test_dataset:
    #input_ids = batch['input_ids']
    #attention_mask = batch['attention_mask']
    #labels = batch['labels']
    #outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
    inputs = batch
    outputs = model(inputs)
    predictions = outputs.logits.argmax(dim=-1)

# 預測
text = "This is a sample text."
input_ids = tokenizer.encode(text, return_tensors='pt')
outputs = model(input_ids)
predictions = outputs.logits.argmax(dim=-1)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

  0%|          | 0/3 [00:00<?, ?it/s]

{'input_ids': tensor([  101,  1045, 12524,  1045,  2572,  8025,  1011,  3756,  2013,  2026,
         2678,  3573,  2138,  1997,  2035,  1996,  6704,  2008,  5129,  2009,
         2043,  2009,  2001,  2034,  2207,  1999,  3476,  1012,  1045,  2036,
         2657,  2008,  2012,  2034,  2009,  2001,  8243,  2011,  1057,  1012,
         1055,  1012,  8205,  2065,  2009,  2412,  2699,  2000,  4607,  2023,
         2406,  1010,  3568,  2108,  1037,  5470,  1997,  3152,  2641,  1000,
         6801,  1000,  1045,  2428,  2018,  2000,  2156,  2023,  2005,  2870,
         1012,  1026,  7987,  1013,  1028,  1026,  7987,  1013,  1028,  1996,
         5436,  2003,  8857,  2105,  1037,  2402,  4467,  3689,  3076,  2315,
        14229,  2040,  4122,  2000,  4553,  2673,  2016,  2064,  2055,  2166,
         1012,  1999,  3327,  2016,  4122,  2000,  3579,  2014,  3086,  2015,
         2000,  2437,  2070,  4066,  1997,  4516,  2006,  2054,  1996,  2779,
        25430, 14728,  2245,  2055,  3056,  2576, 

AttributeError: 'dict' object has no attribute 'size'